### 基于图神经网络的节点表征学习

本节中，我们将学习实现多层图神经网络的方法, 并以节点分类任务为例, 学习训练图神经网络的一般过程
我们将以Cora数据集为例子进行说明, Cora是一个论文引用网络，节点代表论文, 如果两篇论文存在引用关系, 则对应的两个节点之间存在边, 各节点的属性都是一个1433维的词包特征向量。我们的任务是预测各篇论文的类别（共7类）

我们还将对MLP和GCN, GAT（两个知名度很高的图神经网络） 三类神经网络在节点分类任务中的表现进行比较分析，以此来展现图神经网络的强
大和论证图神经网络强于普通深度神经网络的原因。

### 准备工作

1. 获取并分析数据集

In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

# 设置数据集的正则化
dataset = Planetoid(root='dataset', name='Cora', transform=NormalizeFeatures())

In [2]:
from torch_geometric_code.utils.graph_info import getGraphInfo, getDatasetInfo

# 打印数据集信息
getDatasetInfo(dataset)

# 获取第一张图
data = dataset[0]
print(data)
getGraphInfo(data)

数据集 : Cora()
数据集中图的数量 : 1
数据集的特征数量 : 1433
数据集的类别数量 : 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
节点数 : 2708 
边的数量 : 10556 
平均节点的度 : 3.8980797636632203 
可训练的节点数 : 75.4000015258789 
包含的独立节点数 : False 
包含的环形节点数 : False 
是否是无向图 : True 


D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


我们可以看到，Cora图拥有2,708个节点和10,556条边，平均节点度为3.9,
训练集仅使用了140个节点，占整体的5%。我们还可以看到，这个图是无向图，不存在孤立的节点。

数据转换（transform）在将数据输入到神经网络之前修改数据，这一功能可用于 实现数据规范化或数据增强

我们使用NormalizeFeatures进行节点 特征归一化，使各节点特征总和为1。
其他的数据转换方法请参阅[torch-geometric.transforms](https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html#torch-geometric-transforms)。

### 可视化节点表征分布的方法

我们先利用TSNE方法将高维的节点表征映射到二维平面空间，然后在二维平面画出节点，这样我们就实现了节点表征分布的可视化。

In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(out, color):
    """
    :param out: out 是图神经网络模型的参数
    :param color:
    :return:
    """
    z = TSNE(n_components=2)
    z.fit_transform(out.detach().cpu().numpy())
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:, 0], z[:, 1], s = 70, color=color, cmaps = "Set2")
    plt.show()

### 使用MLP神经网络进行节点分类
理论上，我们应该能够仅根据文章的内容，即它的词包特征表征（bag-of-words
feature representation）来推断文章的类别，而无需考虑文章之间的任何关系信息。
接下来, 让我们通过构建一个简单的MLP神经网络来验证这一点。此神经网络只对输入节点的表征做变换，它在所有节点之间共享权重。

我们的MLP由两个线程层、一个ReLU非线性层和一个dropout操作组成。第一个线 程 层 将 1433 维 的 节 点 表 征 嵌 入 (embedding) 到 低 维 空 间 中 (hidden_channels=16), 第 二 个 线 性 层 将 节 点 表 征 嵌 入 到 类 别 空 间 中 (num_classes=7)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, num_features: object, num_classes: object, hidden_channels: object) -> object:
        super(MLP, self).__init__()
        # 设置随机种子
        torch.manual_seed(1234)
        self.lin1 = nn.Linear(num_features, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, num_classes)

    def forward(self, x):
        """
        :param x: x [num_nodes, num_features]
        :return:
        """
        x = self.lin1(x) # [num_nodes, hidden_channels]
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x) # [num_nodes, num_classes]
        return x

model = MLP(dataset.num_features, dataset.num_classes, hidden_channels=16)
print(model)

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


### MLP神经网络的训练
我们利用交叉熵损失和Adam优化器来训练这个简单的MLP神经网络。

In [10]:
# 定义交叉熵损失函数
criterion = nn.CrossEntropyLoss()
# 定义Adam优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

定义训练方法

In [11]:
def train():
    model.train()
    # 清除梯度
    optimizer.zero_grad()
    out = model(data.x)
    # 根据训练节点计算loss值
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    # 反向传播
    loss.backward()
    # 更新梯度
    optimizer.step()
    return loss

In [22]:
from tqdm import tqdm
for epoch in tqdm(range(1, 10)):
    loss = 0
    for item in range(500):
        loss = train()
    print(f'Epoch : {epoch:03d}, Loss : {loss:.4f}')

 11%|█         | 1/9 [00:03<00:26,  3.30s/it]

Epoch : 001, Loss : 0.2440


 22%|██▏       | 2/9 [00:06<00:23,  3.38s/it]

Epoch : 002, Loss : 0.2935


 33%|███▎      | 3/9 [00:10<00:20,  3.33s/it]

...

Epoch : 007, Loss : 0.2613


 89%|████████▉ | 8/9 [00:25<00:03,  3.14s/it]

Epoch : 008, Loss : 0.2314


100%|██████████| 9/9 [00:28<00:00,  3.21s/it]

Epoch : 009, Loss : 0.2539


### MLP神经网络的测试
训练完模型后，我们可以通过测试来检验这个简单的MLP神经网络在测试集上的表现

In [20]:
def test():
    model.eval()
    out = model(data.x) # [num_nodes. num_classes]
    pred = out.argmax(dim=1)
    # 测试集精确度
    test_correct = pred[data.test_mask] == data.y[data.test_mask]
    # acc
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    return test_acc

In [21]:
test_acc = test()
print(f'test accuracy : {test_acc:.4f}')

test accuracy : 0.5350
